In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.options.display.max_rows = 10
pd.options.display.max_columns = 500

In [2]:
###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['ml_outcomes'] = df['ml_outcomes_div'].round(2) #+ df['R']*0.13



In [3]:
len(df)

1000000

In [4]:
FRAC_ADMIT = df[['A']].sum()/len(df)


In [5]:
##
# R = race, T = test score, ml_outcomes = expected utility from admitting, 
# ml_outcomes_{} counterfactual utility given race 
##

df[['R','T','ml_outcomes','T_black','T_white','ml_outcomes_black','ml_outcomes_white','ml_outcomes_decision']].sort_values(by='ml_outcomes')



,R,T,ml_outcomes,T_black,T_white,ml_outcomes_black,ml_outcomes_white,ml_outcomes_decision
799856,0,7,0.02,6,7,-4.079949,-3.784626,0
381520,0,8,0.02,6,8,-4.079949,-3.705448,0
802371,0,12,0.03,10,12,-3.763237,-3.388736,0
475990,0,10,0.03,7,10,-4.000771,-3.547092,0
195611,0,12,0.03,7,12,-4.000771,-3.388736,0
...,...,...,...,...,...,...,...,...
774157,1,106,1.23,106,124,3.837851,5.479200,1
733667,1,108,1.23,108,126,3.996207,5.637556,1
777126,1,104,1.23,104,121,3.679495,5.241666,1
961510,1,105,1.23,105,123,3.758673,5.400022,1


In [6]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','ml_outcomes']].groupby(['R','T']).sum().reset_index()

In [7]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes']].groupby(['R','T']).count().reset_index()
df_count.columns = ['R','T','Count']
df_count['N'] = df_count['Count']

In [8]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [9]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,ml_outcomes,N
0,225,0,0,7,0.02,1
1,224,1,0,8,0.02,1
3,223,3,0,10,0.03,1
4,222,4,0,11,0.03,1
2,221,2,0,9,0.06,2
...,...,...,...,...,...,...
49,4,49,0,56,10957.96,21073
52,3,52,0,59,11026.38,19011
53,2,53,0,60,11038.80,18398
50,1,50,0,57,11087.82,20533


### Setup optimization problem 

In [10]:
from ortools.linear_solver import pywraplp


In [11]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [12]:
dff

,level_0,index,R,T,ml_outcomes,N
0,225,0,0,7,0.02,1
1,224,1,0,8,0.02,1
2,221,2,0,9,0.06,2
3,223,3,0,10,0.03,1
4,222,4,0,11,0.03,1
...,...,...,...,...,...,...
221,202,221,1,105,1.23,1
222,190,222,1,106,3.69,3
223,203,223,1,107,1.23,1
224,197,224,1,108,2.46,2


In [13]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [14]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [15]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total applicants cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

485361


In [16]:
# Now we have one constraint
solver.NumConstraints()

1

## Add CF Fair constraints

In [17]:
from collections import Counter

def convertListToProb(raw_list):
    counts = dict(Counter(raw_list))
    probs = {}
    for test_score in counts:
        probs[test_score]  = counts[test_score]/float(len(raw_list))
    return [(probs[t], t) for t in probs]

In [18]:
T_blacks_list = df[df['R']==0][['T','T_black_star']].groupby('T')['T_black_star'].apply(list).reset_index(name='T_blacks')


In [19]:
T_blacks_list['probs'] = T_blacks_list['T_blacks'].apply(convertListToProb)


In [20]:
T_blacks_list

,T,T_blacks,probs
0,7,[8],"[(1.0, 8)]"
1,8,[8],"[(1.0, 8)]"
2,9,"[9, 9]","[(1.0, 9)]"
3,10,[9],"[(1.0, 9)]"
4,11,[12],"[(1.0, 12)]"
...,...,...,...
117,125,[116],"[(1.0, 116)]"
118,127,[117],"[(1.0, 117)]"
119,128,[118],"[(1.0, 118)]"
120,130,[120],"[(1.0, 120)]"


In [21]:
didntexist = 0
exists = 0
for ix, row in T_blacks_list.iterrows():
    cf_fair_stratum = solver.Constraint(0.0, 0.0)

    White_T = row['T']
    Blacks_Ts = row['probs']
    
    cf_fair_stratum.SetCoefficient(vars_cache[(0.0, White_T)], -1.0)
    for prob in Blacks_Ts:
        if (1.0, prob[1]) not in vars_cache:
            vars_cache[(1.0, prob[1])] = solver.NumVar(0.0, 1.0, str((1.0, prob[1])))
            didntexist+=1
        else:
            exists+=1
        cf_fair_stratum.SetCoefficient(vars_cache[(1.0, prob[1])], prob[0])
    

In [22]:
didntexist,exists

(11, 721)

In [23]:
T_whites_list = df[df['R']==1][['T','T_white_star']].groupby('T')['T_white_star'].apply(list).reset_index(name='T_whites')

In [24]:
T_whites_list['probs'] = T_whites_list['T_whites'].apply(convertListToProb)


In [25]:
didntexist = 0
exists = 0

for ix, row in T_whites_list.iterrows():
    cf_fair_stratum = solver.Constraint(0.0, 0.0)

    Black_T = row['T']
    White_Ts = row['probs']
    
    cf_fair_stratum.SetCoefficient(vars_cache[(1.0, Black_T)], -1.0)
    for prob in White_Ts:
        if (0.0, prob[1]) not in vars_cache:
            vars_cache[(0.0, prob[1])] = solver.NumVar(0.0, 1.0, str((0.0, prob[1])))
            didntexist+=1
        else:
            exists+=1
            
        cf_fair_stratum.SetCoefficient(vars_cache[(0.0, prob[1])], prob[0])
    

In [26]:
cf_fair_stratum.basis_status

<bound method Constraint.basis_status of <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fb9998f5390> >>

In [27]:
solver.NumConstraints()

227

## Solve linear program

In [28]:
solver.ABNORMAL

4

In [29]:
status = solver.Solve()


In [30]:
status

0

In [31]:
solver.OPTIMAL

0

In [32]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [33]:
df_decisions

,row_id,decision
0,0,0.000000
1,1,0.000000
2,2,0.634978
3,3,0.634978
4,4,0.490718
...,...,...
221,221,0.485353
222,222,0.485353
223,223,0.485353
224,224,0.485353


In [34]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,ml_outcomes,N,row_id,decision
51,0,51,0,58,11123.28,19863,51,0.485353
50,1,50,0,57,11087.82,20533,50,0.485353
53,2,53,0,60,11038.80,18398,53,0.485353
52,3,52,0,59,11026.38,19011,52,0.485353
49,4,49,0,56,10957.96,21073,49,0.485353
...,...,...,...,...,...,...,...,...
2,221,2,0,9,0.06,2,2,0.634978
4,222,4,0,11,0.03,1,4,0.490718
3,223,3,0,10,0.03,1,3,0.634978
1,224,1,0,8,0.02,1,1,0.000000


In [35]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,ml_outcomes,N,row_id,decision
51,0,51,0,58,11123.28,19863,51,0.485353
50,1,50,0,57,11087.82,20533,50,0.485353
53,2,53,0,60,11038.80,18398,53,0.485353
52,3,52,0,59,11026.38,19011,52,0.485353
49,4,49,0,56,10957.96,21073,49,0.485353
...,...,...,...,...,...,...,...,...
2,221,2,0,9,0.06,2,2,0.634978
4,222,4,0,11,0.03,1,4,0.490718
3,223,3,0,10,0.03,1,3,0.634978
1,224,1,0,8,0.02,1,1,0.000000


In [36]:
xxx.sort_values(by='decision',ascending=False)

,level_0,index,R,T,ml_outcomes,N,row_id,decision
117,211,117,0,125,1.00,1,117,1.0
114,192,114,0,122,3.00,3,114,1.0
115,199,115,0,123,2.00,2,115,1.0
116,206,116,0,124,1.00,1,116,1.0
121,207,121,0,134,1.00,1,121,1.0
...,...,...,...,...,...,...,...,...
122,219,122,1,4,0.26,1,122,0.0
124,218,124,1,8,0.27,1,124,0.0
123,215,123,1,7,0.54,2,123,0.0
1,224,1,0,8,0.02,1,1,0.0


In [37]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)

In [38]:
admit_decisions = df.merge(xxx,how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT['A']

FRAC_BLACK_POLICY = (admit_decisions['R_y'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R_y'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision_random']).sum()


In [39]:
file = open('./lp_results.csv','a')
file.write('{}\t{}\t{}\n'.format('Path-Specific Fairness',str(FRAC_BLACK_POLICY),str(SUM_BP_POLICY)))
file.close()


In [40]:
(xxx['N']*xxx['decision']).sum()/len(df)

0.48536099999999976